In [38]:
from dotenv import load_dotenv
import os
import glob
import math
import google.generativeai as genai

load_dotenv("/home/harish/Desktop/python/chatbot/.env")
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key) 
model_name = "gemini-1.5-flash"

def chunk_text(text, max_length=500):
    """Splits a single text string into chunks of a max length."""
    return [text[i:i + max_length] for i in range(0, len(text), max_length)]

# 1. FIND ALL FILE PATHS (Your code is good)
doc_path = "/home/harish/Desktop/python/chatbot/docs/**/*" # Using '/*' is slightly more robust
doc_files = glob.glob(doc_path, recursive=True)

# Filter for specific file extensions
doc_files = [f for f in doc_files if f.endswith((".md", ".txt", ".html"))]

if not doc_files:
    raise ValueError("❌ No documents found. Check your docs path and file extensions!")

chunks = []
for f in doc_files:
    with open(f, "r", encoding="utf-8") as file:
        text = file.read()
    for chunk in chunk_text(text):
        embedding = genai.embed_content(
            model="models/text-embedding-004",
            content=chunk
        )
        chunks.append({"text": chunk, "vector": embedding["embedding"]})

if not chunks:
    raise ValueError("❌ No chunks created. Are your documents empty?")

print(f"✅ Loaded {len(chunks)} chunks from {len(doc_files)} documents.")

# Cosine similarity helper
def cosine_similarity(v1, v2):
    dot = sum(a * b for a, b in zip(v1, v2))
    mag1 = math.sqrt(sum(x ** 2 for x in v1))
    mag2 = math.sqrt(sum(x ** 2 for x in v2))
    return dot / (mag1 * mag2)

# Setup Chat Model
chat_model = genai.GenerativeModel(model_name)

# Ask Bot function
def ask_bot(question: str) -> str:
    """Ask the chatbot a question based on your docs."""
    # Embed question
    q_embed = genai.embed_content(
        model="models/text-embedding-004",
        content=question
    )["embedding"]

    # Rank chunks by similarity
    ranked = sorted(
        [(c, cosine_similarity(q_embed, c["vector"])) for c in chunks],
        key=lambda x: -x[1]
    )[:3]

    # Build prompt
    context = "\n---\n".join(c["text"] for c, _ in ranked)
    prompt = f"""
    You are an assistant that answers questions based only on the following document excerpts.
    If the answer isn't clearly in the documents, reply with "I don't know."
    DOCUMENTS:
    {context}
    QUESTION: {question}
    """

    # Ask Gemini
    response = chat_model.generate_content([{"role": "user", "parts": [prompt]}])
    return response.text


✅ Loaded 855 chunks from 83 documents.


In [39]:
ask_bot("explain about aws cli") # this data is given inside my docs 

'The AWS CLI (Command Line Interface) needs to be installed on your local machine or deployment server.  Installation instructions are provided for macOS (using `brew install awscli`), Ubuntu/Debian (using `sudo apt install awscli`), and a cross-platform method using pip (`pip install awscli`).\n'

In [40]:
ask_bot("kya aapke ghar me pintu hai")  # this data is not given inside my docs

"I don't know.\n"

In [41]:
ask_bot("how to create auto scalling group")

'To create an Auto Scaling Group, go to the EC2 Dashboard → Auto Scaling Groups → Create Auto Scaling Group.  Attach the launch template and set the desired, minimum, and maximum instance counts.\n'

In [43]:
ask_bot("do you know worker pettern if yes then explain me about it")

'Yes, the document describes a worker pattern using Sidekiq workers for background job processing in the MGR API.  These workers handle time-consuming operations (data import, email/SMS notifications, file processing, external API integrations, and batch operations) to prevent blocking the main application thread.  The architecture uses Sidekiq integration, and includes structured logging for tracking events.  Testing guidance includes using mocks, real data (via fabricators), authorization checks, and performance testing with realistic data volumes.\n'